In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
# Make sure we're in the right directory
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [5]:
import xarray as xr
import numpy as np
from typing import *
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from aibedo.utilities.plotting import data_snapshots_plotting, data_mean_plotting, zonal_error_plotting
from aibedo.utilities.wandb_api import get_predictions_xarray

In [6]:
predict_set = "same_as_test"  # which data to predict on, can be {<any ESM>, "val", "same_as_test"}, val means the validation-set of the same ESM used for training
test_set = "era5"  # can be 'era5' or 'merra2'. In conjunction with predict_set="same_as_test", predictions will be made on this dataset
DATA_DIR = "./data"

overrides = [f'datamodule.num_workers={0}',  # e.g. set to number of CPUs
             f'datamodule.eval_batch_size=1',  # the lower, the less memory will be needed
             f'verbose=False',
             f'datamodule.data_dir={DATA_DIR}',
             f'datamodule.partition=[0.85, 0.15, {test_set}]',  # partition of train, validation, test fractions/sets
             f'++datamodule.prediction_data={predict_set}', 
             f"++model.use_auxiliary_vars=False",  # needs to be True when training only for computation of physics constraints
            ]

In [7]:
predict_xarray_kwargs = dict(
    also_errors=True,   # whether to return errors in the xarray too (bias and MAE if true), formatted as per 'tas_mae', 'tas_pre_bias' etc.
    also_targets=False,  # whether to return the target data (e.g. ERA5 if testing on ERA5), formatted as per 'tas_targets', 'tas_pre_targets' etc.\
    split='predict', 
    variables='all',    # which variable to return predictions for. Can be "all" or any subset of ['pr', 'ps', 'tas']
    return_normalized_outputs=False,  # if true: also return predictions (and targets/errors) for the normalized output variables (with _pre suffix)
)

# Reload and predict with some models:

In [8]:
rid_fno_1d = "segysmbc"
ds_fno1d = get_predictions_xarray(rid_fno_1d, overrides, return_model=False, **predict_xarray_kwargs)

In [ ]:
rid_mlp_1d = "8hnsi0cn"
ds_mlp1d = get_predictions_xarray(rid_mlp_1d, overrides, return_model=False, **predict_xarray_kwargs)

In [ ]:
ds_mlp1d

In [ ]:
plt.rcParams['figure.figsize'] = [40, 15]  # general matplotlib parameters
_ = zonal_error_plotting([ds_mlp1d, ds_fno1d], labels=["MLP", 'FNO'],
                         latitude_dim='latitude_grouped', error_to_plot='mae')  # error_to_plot can be 'bias' or 'mae'